## Identifying and resolving conflicts in a batch of VI files

ELGs - part 9 is sets 41 to 45

736 visual inspections of a total of 250 unique objects

38 conflicts

In [1]:
save_filename='truth_table_ELGs_sets41_45.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_41_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_41_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_41_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_42_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_42_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_42_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_43_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_43_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_43_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_44_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_44_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_44_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_45_SAE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_45_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_45_TWL.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_41_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_41_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_42_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_42_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_42_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_43_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_43_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_43_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_44_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_44_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_44_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_45_SAE.csv
desi-vi_SV0_ELG_tile67230_night20200315_45_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_45_TWL.csv
There are 736 visual inspections of a total of 250 unique objects
That gives 2.944 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
736


### Adding a bunch of useful columns

In [11]:
vi = VI.prep_new_columns(vi)

In [12]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185790258512016,-1,0.33.0.dev3890,0,GALAXY,1.0865,TWL,3,--,--,...,150.555734,1.0865,GALAXY,3.333333,1,0.000000,1.086500,[OII] on sky lines|--|--,3,none
1,35185790258512067,-1,0.33.0.dev3890,0,GALAXY,0.1621,TWL,4,--,--,...,819.743441,0.1621,GALAXY,4.000000,0,0.000000,0.162100,--|--|--,3,none
2,35185790258512077,-1,0.33.0.dev3890,0,GALAXY,1.4432,TWL,0,--,--,...,1.433890,1.4432,GALAXY,0.666667,1,0.000000,1.443200,No obvious features| No clear match to featur...,3,none
3,35185790258512108,-1,0.33.0.dev3890,0,GALAXY,0.8418,TWL,4,--,--,...,985.639847,0.8418,GALAXY,4.000000,0,0.000000,0.841800,Broad [OII]| MgII in absorption.|--,3,none
4,35185790258512631,-1,0.33.0.dev3890,0,GALAXY,1.0344,TWL,4,--,--,...,811.907264,1.0344,GALAXY,4.000000,0,0.000000,1.034400,Triple peak [OII]|--|--,3,none
5,35185790258512788,-1,0.33.0.dev3890,0,GALAXY,1.4882,TWL,0,--,--,...,0.980285,1.4882,GALAXY,1.000000,2,0.000000,1.488200,No obvious features| No match to weak features...,3,none
6,35185790258513245,-1,0.33.0.dev3890,0,GALAXY,0.9412,TWL,4,--,--,...,1616.551816,0.9412,GALAXY,4.000000,0,0.000000,0.941200,Broad [OII]|--|--,3,none
7,35185790258513304,-1,0.33.0.dev3890,0,GALAXY,1.2098,TWL,4,--,--,...,182.209876,1.2098,GALAXY,3.666667,1,0.000000,1.209800,Triple peak [OII]|--|--,3,none
8,35185790258513374,-1,0.33.0.dev3890,0,GALAXY,1.3557,TWL,2,--,--,...,28.631490,1.3557,GALAXY,3.000000,2,0.000212,1.355867,Low S/N emission line features|--|--,3,none
9,35185790258513897,-1,0.33.0.dev3890,0,GALAXY,0.7995,TWL,4,--,--,...,884.435287,0.7995,GALAXY,4.000000,0,0.000000,0.799500,--|--|--,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [13]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [14]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185784260661713 35185784264852626 35185784264853084 35185784264855191
 35185790258513374 35185790258514059 35185790262708209 35185790262710484
 35185790266900847 35185790266904796 35185790271095505 35185790271097088
 35185790271097883 35185790279485917 35185790279486104 35185790279486733
 35185790279486934 35185790279487110 35185790279487153 35185790279488094
 35185790279488374 35185790283677815 35185790283677860 35185790283678333
 35185790283678769 35185790283679657 35185790283681353 35185790283682341
 35185790283683134 35185790287872074 35185790287872604 35185790292066750
 35185790292071269 35185790296260657 35185790296262065 35185790296263776
 35185790296263840 35185790296264034]
Total number of conflicts to resolve:  38


In [15]:
unique_targets[1]

35185784264852626

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [16]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [17]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [18]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
22,35185784260661713,GALAXY,1.1575,TWL,-1,--,--,--,Triple peak [OII],1.1575,GALAXY,2.333333,5,1.1575,0.0,Triple peak [OII]|--|--,3,none
72,35185784260661713,GALAXY,1.1575,ACE,4,--,--,--,--,1.1575,GALAXY,2.333333,5,1.1575,0.0,Triple peak [OII]|--|--,3,none
122,35185784260661713,GALAXY,1.1575,CGS,4,--,--,--,--,1.1575,GALAXY,2.333333,5,1.1575,0.0,Triple peak [OII]|--|--,3,none


In [19]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'One inspector forgot to set the flag. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
22,35185784260661713,-1,0.33.0.dev3890,0,GALAXY,1.1575,TWL,4,--,--,...,634.645283,1.1575,GALAXY,2.333333,5,0.0,1.1575,Triple peak [OII]|--|--,3,One inspector forgot to set the flag. z is sec...
72,35185784260661713,-1,0.33.0.dev3890,0,GALAXY,1.1575,ACE,4,--,--,...,634.645283,1.1575,GALAXY,2.333333,5,0.0,1.1575,Triple peak [OII]|--|--,3,One inspector forgot to set the flag. z is sec...
122,35185784260661713,-1,0.33.0.dev3890,0,GALAXY,1.1575,CGS,4,--,--,...,634.645283,1.1575,GALAXY,2.333333,5,0.0,1.1575,Triple peak [OII]|--|--,3,One inspector forgot to set the flag. z is sec...


conflict ID =  0


In [20]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
31,35185784264852626,GALAXY,0.7687,TWL,0,S,--,--,Model is driven by calibration features,0.7687,GALAXY,1.333333,3,0.7687,0.015379,Model is driven by calibration features| Weak...,3,none
81,35185784264852626,GALAXY,0.7687,ACE,3,R,0.7959,GALAXY,Weak OII and Hgamma and Hbeta,0.7959,GALAXY,1.333333,3,0.7687,0.015379,Model is driven by calibration features| Weak...,3,none
131,35185784264852626,GALAXY,0.7687,CGS,1,--,--,--,--,0.7687,GALAXY,1.333333,3,0.7687,0.015379,Model is driven by calibration features| Weak...,3,none


In [21]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7687
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no clear features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
31,35185784264852626,-1,0.33.0.dev3890,0,GALAXY,0.7687,TWL,1,S,--,...,0.099587,0.7687,GALAXY,1.333333,3,0.015379,0.7687,Model is driven by calibration features| Weak...,3,disagreement in flag and z. no clear features....
81,35185784264852626,-1,0.33.0.dev3890,0,GALAXY,0.7687,ACE,1,R,0.7959,...,0.099587,0.7687,GALAXY,1.333333,3,0.015379,0.7687,Model is driven by calibration features| Weak...,3,disagreement in flag and z. no clear features....
131,35185784264852626,-1,0.33.0.dev3890,0,GALAXY,0.7687,CGS,1,--,--,...,0.099587,0.7687,GALAXY,1.333333,3,0.015379,0.7687,Model is driven by calibration features| Weak...,3,disagreement in flag and z. no clear features....


conflict ID =  1


In [22]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
33,35185784264853084,GALAXY,0.7613,TWL,0,--,--,--,No obvious features,0.7613,GALAXY,1.333333,3,0.7613,1.172884,No obvious features| Very bright Lyman alpha? ...,3,none
83,35185784264853084,GALAXY,0.7613,ACE,3,R,2.8271,GALAXY,Very bright Lyman alpha? Appears to be a doub...,2.8271,GALAXY,1.333333,3,0.7613,1.172884,No obvious features| Very bright Lyman alpha? ...,3,none
133,35185784264853084,GALAXY,0.7613,CGS,1,--,--,--,--,0.7613,GALAXY,1.333333,3,0.7613,1.172884,No obvious features| Very bright Lyman alpha? ...,3,none


In [23]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2485
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'very strong line, unsure of what it might be. Double peaked but profile and delta_lambda does not match OII very well. Double-peaked LyA? Not very broad. No continuum. Take z of OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
33,35185784264853084,-1,0.33.0.dev3890,0,GALAXY,0.7613,TWL,2,--,--,...,10.316831,0.2485,GALAXY,1.333333,3,1.172884,0.7613,No obvious features| Very bright Lyman alpha? ...,3,"very strong line, unsure of what it might be. ..."
83,35185784264853084,-1,0.33.0.dev3890,0,GALAXY,0.7613,ACE,2,R,2.8271,...,10.316831,0.2485,GALAXY,1.333333,3,1.172884,0.7613,No obvious features| Very bright Lyman alpha? ...,3,"very strong line, unsure of what it might be. ..."
133,35185784264853084,-1,0.33.0.dev3890,0,GALAXY,0.7613,CGS,2,--,--,...,10.316831,0.2485,GALAXY,1.333333,3,1.172884,0.7613,No obvious features| Very bright Lyman alpha? ...,3,"very strong line, unsure of what it might be. ..."


conflict ID =  2


In [24]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185784264855191,GALAXY,0.557,TWL,2,--,--,--,Not sure the model is driven by sky lines,0.557,GALAXY,3.333333,2,0.557,0.0,Not sure the model is driven by sky lines|--|--,3,none
92,35185784264855191,GALAXY,0.557,ACE,4,R,--,--,--,0.557,GALAXY,3.333333,2,0.557,0.0,Not sure the model is driven by sky lines|--|--,3,none
142,35185784264855191,GALAXY,0.557,CGS,4,--,--,--,--,0.557,GALAXY,3.333333,2,0.557,0.0,Not sure the model is driven by sky lines|--|--,3,none


In [25]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. OIII and Hb on sky lines, but enough signal to believe them.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
42,35185784264855191,-1,0.33.0.dev3890,0,GALAXY,0.557,TWL,3.333333,--,--,...,44.562699,0.557,GALAXY,3.333333,2,0.0,0.557,Not sure the model is driven by sky lines|--|--,3,"flag disagreement, take mean. OIII and Hb on s..."
92,35185784264855191,-1,0.33.0.dev3890,0,GALAXY,0.557,ACE,3.333333,R,--,...,44.562699,0.557,GALAXY,3.333333,2,0.0,0.557,Not sure the model is driven by sky lines|--|--,3,"flag disagreement, take mean. OIII and Hb on s..."
142,35185784264855191,-1,0.33.0.dev3890,0,GALAXY,0.557,CGS,3.333333,--,--,...,44.562699,0.557,GALAXY,3.333333,2,0.0,0.557,Not sure the model is driven by sky lines|--|--,3,"flag disagreement, take mean. OIII and Hb on s..."


conflict ID =  3


In [26]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
8,35185790258513374,GALAXY,1.3557,TWL,2.0,--,--,--,Low S/N emission line features,1.3557,GALAXY,3.0,2,1.355867,0.000212,Low S/N emission line features|--|--,3,none
58,35185790258513374,GALAXY,1.3557,ACE,3.0,--,--,--,--,1.3557,GALAXY,3.0,2,1.355867,0.000212,Low S/N emission line features|--|--,3,none
108,35185790258513374,GALAXY,1.3557,CGS,4.0,R,1.3562,--,--,1.3562,GALAXY,3.0,2,1.355867,0.000212,Low S/N emission line features|--|--,3,none


In [27]:

#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. Weak line, but continuum consistent with OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
8,35185790258513374,-1,0.33.0.dev3890,0,GALAXY,1.3557,TWL,3.0,--,--,...,28.63149,1.3557,GALAXY,3.0,2,0.000212,1.355867,Low S/N emission line features|--|--,3,"flag disagreement, take mean. Weak line, but c..."
58,35185790258513374,-1,0.33.0.dev3890,0,GALAXY,1.3557,ACE,3.0,--,--,...,28.63149,1.3557,GALAXY,3.0,2,0.000212,1.355867,Low S/N emission line features|--|--,3,"flag disagreement, take mean. Weak line, but c..."
108,35185790258513374,-1,0.33.0.dev3890,0,GALAXY,1.3557,CGS,3.0,R,1.3562,...,28.63149,1.3562,GALAXY,3.0,2,0.000212,1.355867,Low S/N emission line features|--|--,3,"flag disagreement, take mean. Weak line, but c..."


conflict ID =  4


In [28]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
11,35185790258514059,GALAXY,1.2823,TWL,0.0,--,--,--,No obvious features; calibration features,1.2823,GALAXY,2.333333,4,1.2823,0.246477,No obvious features; calibration features|--|...,3,none
61,35185790258514059,GALAXY,1.2823,ACE,4.0,R,0.8310,GALAXY,--,0.8310,GALAXY,2.333333,4,1.2823,0.246477,No obvious features; calibration features|--|...,3,none
111,35185790258514059,GALAXY,1.2823,CGS,3.0,RC,0.8319,--,bad fit? - jump in spectrum,0.8319,GALAXY,2.333333,4,1.2823,0.246477,No obvious features; calibration features|--|...,3,none


In [29]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (0.8319 + 0.8310)/2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors give same VI z. Unresolved OII but also OIII and some abs features.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
11,35185790258514059,-1,0.33.0.dev3890,0,GALAXY,1.2823,TWL,3.0,--,--,...,9.856704,0.83145,GALAXY,2.333333,4,0.246477,1.2823,No obvious features; calibration features|--|...,3,disagreement in flag and z. 2/3 inspectors giv...
61,35185790258514059,-1,0.33.0.dev3890,0,GALAXY,1.2823,ACE,3.0,R,0.8310,...,9.856704,0.83145,GALAXY,2.333333,4,0.246477,1.2823,No obvious features; calibration features|--|...,3,disagreement in flag and z. 2/3 inspectors giv...
111,35185790258514059,-1,0.33.0.dev3890,0,GALAXY,1.2823,CGS,3.0,RC,0.8319,...,9.856704,0.83145,GALAXY,2.333333,4,0.246477,1.2823,No obvious features; calibration features|--|...,3,disagreement in flag and z. 2/3 inspectors giv...


conflict ID =  5


In [30]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
156,35185790262708209,GALAXY,1.3394,TWL,0.0,--,--,--,No obvious features,1.3394,GALAXY,2.666667,4,1.3394,0.260724,No obvious features| OII and OIII clear.|reds...,3,none
206,35185790262708209,GALAXY,1.3394,ACE,4.0,R,0.8558,GALAXY,OII and OIII clear.,0.8558,GALAXY,2.666667,4,1.3394,0.260724,No obvious features| OII and OIII clear.|reds...,3,none
256,35185790262708209,GALAXY,1.3394,SAE,4.0,R,0.8556,GALAXY,redshift fit is confused by low flux and calib...,0.8556,GALAXY,2.666667,4,1.3394,0.260724,No obvious features| OII and OIII clear.|reds...,3,none


In [31]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8557
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z based on OII/OIII. Secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
156,35185790262708209,-1,0.33.0.dev3890,0,GALAXY,1.3394,TWL,3.0,--,--,...,3.20774,0.8557,GALAXY,2.666667,4,0.260724,1.3394,No obvious features| OII and OIII clear.|reds...,3,2/3 inspectors give same VI z based on OII/OII...
206,35185790262708209,-1,0.33.0.dev3890,0,GALAXY,1.3394,ACE,3.0,R,0.8558,...,3.20774,0.8557,GALAXY,2.666667,4,0.260724,1.3394,No obvious features| OII and OIII clear.|reds...,3,2/3 inspectors give same VI z based on OII/OII...
256,35185790262708209,-1,0.33.0.dev3890,0,GALAXY,1.3394,SAE,3.0,R,0.8556,...,3.20774,0.8557,GALAXY,2.666667,4,0.260724,1.3394,No obvious features| OII and OIII clear.|reds...,3,2/3 inspectors give same VI z based on OII/OII...


conflict ID =  6


In [32]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
174,35185790262710484,QSO,4.4968,TWL,3.0,R,1.0597,--,Calibration features drive the best fit,1.0597,QSO,3.0,0,1.0597,0.0,Calibration features drive the best fit| Cont...,3,none
224,35185790262710484,QSO,4.4968,ACE,3.0,RC,1.0597,GALAXY,"Continuum """"hump"""" at 6800-7600A",1.0597,GALAXY,3.0,0,1.0597,0.0,Calibration features drive the best fit| Cont...,3,none
274,35185790262710484,QSO,4.4968,SAE,3.0,RC,1.0597,GALAXY,unusual flux bump confused the fit(foreground ...,1.0597,GALAXY,3.0,0,1.0597,0.0,Calibration features drive the best fit| Cont...,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype, one inspector forgot to change to GALAXY'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
174,35185790262710484,-1,0.33.0.dev3890,0,QSO,4.4968,TWL,3.0,R,1.0597,...,107.083141,1.0597,GALAXY,3.0,0,0.0,1.0597,Calibration features drive the best fit| Cont...,3,"disagreement in spectype, one inspector forgot..."
224,35185790262710484,-1,0.33.0.dev3890,0,QSO,4.4968,ACE,3.0,RC,1.0597,...,107.083141,1.0597,GALAXY,3.0,0,0.0,1.0597,Calibration features drive the best fit| Cont...,3,"disagreement in spectype, one inspector forgot..."
274,35185790262710484,-1,0.33.0.dev3890,0,QSO,4.4968,SAE,3.0,RC,1.0597,...,107.083141,1.0597,GALAXY,3.0,0,0.0,1.0597,Calibration features drive the best fit| Cont...,3,"disagreement in spectype, one inspector forgot..."


conflict ID =  7


In [34]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
175,35185790266900847,QSO,1.6662,TWL,0.0,S,--,--,Calibration features,1.6662,QSO,2.333333,4,1.6662,0.611192,"Calibration features|Continuum """"hump"""" at 68...",3,none
225,35185790266900847,QSO,1.6662,ACE,4.0,--,0.8229,GALAXY,"Continuum """"hump"""" at 6800-7600A. Two redshift...",0.8229,GALAXY,2.333333,4,1.6662,0.611192,"Calibration features|Continuum """"hump"""" at 68...",3,none
275,35185790266900847,QSO,1.6662,SAE,3.0,RC,0.6548,GALAXY,flux bump at 6600-7600AA,0.6548,GALAXY,2.333333,4,1.6662,0.611192,"Calibration features|Continuum """"hump"""" at 68...",3,none


In [35]:
print(vi.loc[ 225]['VI comment'])


Continuum ""hump"" at 6800-7600A. Two redshifts from OII 0.8229 and 0.9366.


In [36]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8229
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3 different VI redshifts offered. Take most convinging, flag 2. Calibration issues. Not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
175,35185790266900847,-1,0.33.0.dev3890,0,QSO,1.6662,TWL,2.0,S,--,...,20.546743,0.8229,GALAXY,2.333333,4,0.611192,1.6662,"Calibration features|Continuum """"hump"""" at 68...",3,3 different VI redshifts offered. Take most co...
225,35185790266900847,-1,0.33.0.dev3890,0,QSO,1.6662,ACE,2.0,--,0.8229,...,20.546743,0.8229,GALAXY,2.333333,4,0.611192,1.6662,"Calibration features|Continuum """"hump"""" at 68...",3,3 different VI redshifts offered. Take most co...
275,35185790266900847,-1,0.33.0.dev3890,0,QSO,1.6662,SAE,2.0,RC,0.6548,...,20.546743,0.8229,GALAXY,2.333333,4,0.611192,1.6662,"Calibration features|Continuum """"hump"""" at 68...",3,3 different VI redshifts offered. Take most co...


conflict ID =  8


In [37]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
179,35185790266904796,QSO,4.5233,TWL,2.0,RCS,1.3278,GALAXY,Possible [OII] at z~1.3278; Calibration problem,1.3278,GALAXY,2.666667,2,4.5233,1.372857,Possible [OII] at z~1.3278; Calibration probl...,3,none
229,35185790266904796,QSO,4.5233,ACE,4.0,R,1.3277,GALAXY,"Continuum """"hump"""" at 6800-7600A.",1.3277,GALAXY,2.666667,2,4.5233,1.372857,Possible [OII] at z~1.3278; Calibration probl...,3,none
279,35185790266904796,QSO,4.5233,SAE,2.0,RC,--,GALAXY,there is strong emission likely to be OII but ...,4.5233,GALAXY,2.666667,2,4.5233,1.372857,Possible [OII] at z~1.3278; Calibration probl...,3,none


In [38]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3278
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give VI z based on secure OII. Take with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
179,35185790266904796,-1,0.33.0.dev3890,0,QSO,4.5233,TWL,3.0,RCS,1.3278,...,9.48978,1.3278,GALAXY,2.666667,2,1.372857,4.5233,Possible [OII] at z~1.3278; Calibration probl...,3,disagreement in z and flag. 2/3 inspectors giv...
229,35185790266904796,-1,0.33.0.dev3890,0,QSO,4.5233,ACE,3.0,R,1.3277,...,9.48978,1.3278,GALAXY,2.666667,2,1.372857,4.5233,Possible [OII] at z~1.3278; Calibration probl...,3,disagreement in z and flag. 2/3 inspectors giv...
279,35185790266904796,-1,0.33.0.dev3890,0,QSO,4.5233,SAE,3.0,RC,--,...,9.48978,1.3278,GALAXY,2.666667,2,1.372857,4.5233,Possible [OII] at z~1.3278; Calibration probl...,3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  9


In [39]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
181,35185790271095505,QSO,1.6947,TWL,0.0,--,--,--,Bad calibration,1.6947,QSO,2.0,4,1.6947,0.08934,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,none
231,35185790271095505,QSO,1.6947,ACE,4.0,RC,1.4737,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock mis...",1.4737,GALAXY,2.0,4,1.6947,0.08934,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,none
281,35185790271095505,QSO,1.6947,SAE,2.0,RCS,1.4739,GALAXY,bad calibration confused the fit; the suggeste...,1.4739,GALAXY,2.0,4,1.6947,0.08934,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,none


In [40]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4738
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give same z based on secure OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
181,35185790271095505,-1,0.33.0.dev3890,0,QSO,1.6947,TWL,3.0,--,--,...,15.877009,1.4738,GALAXY,2.0,4,0.08934,1.6947,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,disagreement in z and flag. 2/3 inspectors giv...
231,35185790271095505,-1,0.33.0.dev3890,0,QSO,1.6947,ACE,3.0,RC,1.4737,...,15.877009,1.4738,GALAXY,2.0,4,0.08934,1.6947,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,disagreement in z and flag. 2/3 inspectors giv...
281,35185790271095505,-1,0.33.0.dev3890,0,QSO,1.6947,SAE,3.0,RCS,1.4739,...,15.877009,1.4738,GALAXY,2.0,4,0.08934,1.6947,"Bad calibration| Continuum """"hump"""" at 6800-7...",3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  10


In [41]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
187,35185790271097088,GALAXY,0.7561,TWL,2.0,RS,1.3588,--,Possible [OII] at z~1.3588; Calibration problem,1.3588,GALAXY,2.666667,2,0.7561,0.343261,Possible [OII] at z~1.3588; Calibration probl...,3,none
237,35185790271097088,GALAXY,0.7561,ACE,4.0,R,1.3589,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.3589,GALAXY,2.666667,2,0.7561,0.343261,Possible [OII] at z~1.3588; Calibration probl...,3,none
287,35185790271097088,GALAXY,0.7561,SAE,2.0,R,--,--,flux bump at 6800-7600AA; strong emissions lik...,0.7561,GALAXY,2.666667,2,0.7561,0.343261,Possible [OII] at z~1.3588; Calibration probl...,3,none


In [42]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3589
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors give same VI z based on non-secure OII. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
187,35185790271097088,-1,0.33.0.dev3890,0,GALAXY,0.7561,TWL,2.0,RS,1.3588,...,64.921167,1.3589,GALAXY,2.666667,2,0.343261,0.7561,Possible [OII] at z~1.3588; Calibration probl...,3,disagreement in flag and z. 2/3 inspectors giv...
237,35185790271097088,-1,0.33.0.dev3890,0,GALAXY,0.7561,ACE,2.0,R,1.3589,...,64.921167,1.3589,GALAXY,2.666667,2,0.343261,0.7561,Possible [OII] at z~1.3588; Calibration probl...,3,disagreement in flag and z. 2/3 inspectors giv...
287,35185790271097088,-1,0.33.0.dev3890,0,GALAXY,0.7561,SAE,2.0,R,--,...,64.921167,1.3589,GALAXY,2.666667,2,0.343261,0.7561,Possible [OII] at z~1.3588; Calibration probl...,3,disagreement in flag and z. 2/3 inspectors giv...


conflict ID =  11


In [43]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
188,35185790271097883,QSO,1.7496,TWL,0.0,S,--,--,Calibration problem,1.7496,QSO,1.333333,3,1.7496,0.446319,"Calibration problem|Continuum """"hump"""" at 6800...",3,none
238,35185790271097883,QSO,1.7496,ACE,3.0,R,2.9768,QSO,"Continuum """"hump"""" at 6800-7600A. Redshift fr...",2.9768,QSO,1.333333,3,1.7496,0.446319,"Calibration problem|Continuum """"hump"""" at 6800...",3,none
288,35185790271097883,QSO,1.7496,SAE,1.0,R,--,--,--,1.7496,QSO,1.333333,3,1.7496,0.446319,"Calibration problem|Continuum """"hump"""" at 6800...",3,none


In [44]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.7496
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no secure features. calibration issues. Take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
188,35185790271097883,-1,0.33.0.dev3890,0,QSO,1.7496,TWL,1.0,S,--,...,5.177809,1.7496,QSO,1.333333,3,0.446319,1.7496,"Calibration problem|Continuum """"hump"""" at 6800...",3,disagreement in flag. no secure features. cali...
238,35185790271097883,-1,0.33.0.dev3890,0,QSO,1.7496,ACE,1.0,R,2.9768,...,5.177809,1.7496,QSO,1.333333,3,0.446319,1.7496,"Calibration problem|Continuum """"hump"""" at 6800...",3,disagreement in flag. no secure features. cali...
288,35185790271097883,-1,0.33.0.dev3890,0,QSO,1.7496,SAE,1.0,R,--,...,5.177809,1.7496,QSO,1.333333,3,0.446319,1.7496,"Calibration problem|Continuum """"hump"""" at 6800...",3,disagreement in flag. no secure features. cali...


conflict ID =  12


In [45]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
297,35185790279485917,GALAXY,1.5999,SAE,0.0,R,--,GALAXY,--,1.5999,GALAXY,2.0,3,1.5994,0.000308,--| MgII in absorption?| [OII] next to sky lines,3,none
347,35185790279485917,GALAXY,1.5999,ACE,3.0,--,1.5992,GALAXY,MgII in absorption?,1.5992,GALAXY,2.0,3,1.5994,0.000308,--| MgII in absorption?| [OII] next to sky lines,3,none
397,35185790279485917,GALAXY,1.5999,TWL,3.0,--,1.5991,--,[OII] next to sky lines,1.5991,GALAXY,2.0,3,1.5994,0.000308,--| MgII in absorption?| [OII] next to sky lines,3,none


In [46]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
297,35185790279485917,-1,0.33.0.dev3890,0,GALAXY,1.5999,SAE,2.0,R,--,...,62.617381,1.5999,GALAXY,2.0,3,0.000308,1.5994,--| MgII in absorption?| [OII] next to sky lines,3,"flag disagreement, take mean. z not secure."
347,35185790279485917,-1,0.33.0.dev3890,0,GALAXY,1.5999,ACE,2.0,--,1.5992,...,62.617381,1.5992,GALAXY,2.0,3,0.000308,1.5994,--| MgII in absorption?| [OII] next to sky lines,3,"flag disagreement, take mean. z not secure."
397,35185790279485917,-1,0.33.0.dev3890,0,GALAXY,1.5999,TWL,2.0,--,1.5991,...,62.617381,1.5991,GALAXY,2.0,3,0.000308,1.5994,--| MgII in absorption?| [OII] next to sky lines,3,"flag disagreement, take mean. z not secure."


conflict ID =  13


In [47]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
299,35185790279486104,GALAXY,1.4724,SAE,0.0,R,--,GALAXY,--,1.4724,GALAXY,2.333333,4,1.472133,0.000324,--| MgII in absorption| Triple peak [OII],3,none
349,35185790279486104,GALAXY,1.4724,ACE,3.0,--,1.4716,GALAXY,MgII in absorption,1.4716,GALAXY,2.333333,4,1.472133,0.000324,--| MgII in absorption| Triple peak [OII],3,none
399,35185790279486104,GALAXY,1.4724,TWL,4.0,--,--,--,Triple peak [OII],1.4724,GALAXY,2.333333,4,1.472133,0.000324,--| MgII in absorption| Triple peak [OII],3,none


In [48]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z is secured by OII + continuum. flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
299,35185790279486104,-1,0.33.0.dev3890,0,GALAXY,1.4724,SAE,3.0,R,--,...,80.798372,1.4724,GALAXY,2.333333,4,0.000324,1.472133,--| MgII in absorption| Triple peak [OII],3,"disagreement in flag, but z is secured by OII ..."
349,35185790279486104,-1,0.33.0.dev3890,0,GALAXY,1.4724,ACE,3.0,--,1.4716,...,80.798372,1.4716,GALAXY,2.333333,4,0.000324,1.472133,--| MgII in absorption| Triple peak [OII],3,"disagreement in flag, but z is secured by OII ..."
399,35185790279486104,-1,0.33.0.dev3890,0,GALAXY,1.4724,TWL,3.0,--,--,...,80.798372,1.4724,GALAXY,2.333333,4,0.000324,1.472133,--| MgII in absorption| Triple peak [OII],3,"disagreement in flag, but z is secured by OII ..."


conflict ID =  14


In [49]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
300,35185790279486733,GALAXY,1.5537,SAE,0.0,R,--,GALAXY,--,1.5537,GALAXY,2.333333,4,1.5537,0.0,--|--| [OII] next to sky lines,3,none
350,35185790279486733,GALAXY,1.5537,ACE,3.0,--,--,--,--,1.5537,GALAXY,2.333333,4,1.5537,0.0,--|--| [OII] next to sky lines,3,none
400,35185790279486733,GALAXY,1.5537,TWL,4.0,--,--,--,[OII] next to sky lines,1.5537,GALAXY,2.333333,4,1.5537,0.0,--|--| [OII] next to sky lines,3,none


In [50]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. doublet near sky lines but it is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
300,35185790279486733,-1,0.33.0.dev3890,0,GALAXY,1.5537,SAE,3.0,R,--,...,48.824097,1.5537,GALAXY,2.333333,4,0.0,1.5537,--|--| [OII] next to sky lines,3,disagreement in flag. doublet near sky lines b...
350,35185790279486733,-1,0.33.0.dev3890,0,GALAXY,1.5537,ACE,3.0,--,--,...,48.824097,1.5537,GALAXY,2.333333,4,0.0,1.5537,--|--| [OII] next to sky lines,3,disagreement in flag. doublet near sky lines b...
400,35185790279486733,-1,0.33.0.dev3890,0,GALAXY,1.5537,TWL,3.0,--,--,...,48.824097,1.5537,GALAXY,2.333333,4,0.0,1.5537,--|--| [OII] next to sky lines,3,disagreement in flag. doublet near sky lines b...


conflict ID =  15


In [51]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
301,35185790279486934,GALAXY,0.9992,SAE,0.0,--,--,--,--,0.9992,GALAXY,2.333333,4,0.9992,0.0,--|--|--,3,none
351,35185790279486934,GALAXY,0.9992,ACE,3.0,--,--,--,--,0.9992,GALAXY,2.333333,4,0.9992,0.0,--|--|--,3,none
401,35185790279486934,GALAXY,0.9992,TWL,4.0,--,--,--,--,0.9992,GALAXY,2.333333,4,0.9992,0.0,--|--|--,3,none


In [52]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
301,35185790279486934,-1,0.33.0.dev3890,0,GALAXY,0.9992,SAE,3.0,--,--,...,121.912572,0.9992,GALAXY,2.333333,4,0.0,0.9992,--|--|--,3,"disagreement in flag, but doublet secures z."
351,35185790279486934,-1,0.33.0.dev3890,0,GALAXY,0.9992,ACE,3.0,--,--,...,121.912572,0.9992,GALAXY,2.333333,4,0.0,0.9992,--|--|--,3,"disagreement in flag, but doublet secures z."
401,35185790279486934,-1,0.33.0.dev3890,0,GALAXY,0.9992,TWL,3.0,--,--,...,121.912572,0.9992,GALAXY,2.333333,4,0.0,0.9992,--|--|--,3,"disagreement in flag, but doublet secures z."


conflict ID =  16


In [53]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35185790279487110,GALAXY,1.3819,SAE,0.0,--,--,--,--,1.3819,GALAXY,2.666667,4,1.3819,0.0,--|--|--,3,none
353,35185790279487110,GALAXY,1.3819,ACE,4.0,--,--,--,--,1.3819,GALAXY,2.666667,4,1.3819,0.0,--|--|--,3,none
403,35185790279487110,GALAXY,1.3819,TWL,4.0,--,--,--,--,1.3819,GALAXY,2.666667,4,1.3819,0.0,--|--|--,3,none


In [54]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. doublet is weak, but it is there. take mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35185790279487110,-1,0.33.0.dev3890,0,GALAXY,1.3819,SAE,2.666667,--,--,...,71.501738,1.3819,GALAXY,2.666667,4,0.0,1.3819,--|--|--,3,"disagreement in flag. doublet is weak, but it ..."
353,35185790279487110,-1,0.33.0.dev3890,0,GALAXY,1.3819,ACE,2.666667,--,--,...,71.501738,1.3819,GALAXY,2.666667,4,0.0,1.3819,--|--|--,3,"disagreement in flag. doublet is weak, but it ..."
403,35185790279487110,-1,0.33.0.dev3890,0,GALAXY,1.3819,TWL,2.666667,--,--,...,71.501738,1.3819,GALAXY,2.666667,4,0.0,1.3819,--|--|--,3,"disagreement in flag. doublet is weak, but it ..."


conflict ID =  17


In [55]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
304,35185790279487153,GALAXY,1.6284,SAE,0.0,--,--,--,--,1.6284,GALAXY,1.0,3,1.6284,0.315173,--| Based on Lyman alpha| Model fits sky resid...,3,none
354,35185790279487153,GALAXY,1.6284,ACE,3.0,R,2.4568,QSO,Based on Lyman alpha,2.4568,QSO,1.0,3,1.6284,0.315173,--| Based on Lyman alpha| Model fits sky resid...,3,none
404,35185790279487153,GALAXY,1.6284,TWL,0.0,--,--,--,Model fits sky residuals,1.6284,GALAXY,1.0,3,1.6284,0.315173,--| Based on Lyman alpha| Model fits sky resid...,3,none


In [56]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6284
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. too noisy. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
304,35185790279487153,-1,0.33.0.dev3890,0,GALAXY,1.6284,SAE,1.0,--,--,...,4.175711,1.6284,GALAXY,1.0,3,0.315173,1.6284,--| Based on Lyman alpha| Model fits sky resid...,3,disagreement in all. too noisy. take RR answer...
354,35185790279487153,-1,0.33.0.dev3890,0,GALAXY,1.6284,ACE,1.0,R,2.4568,...,4.175711,1.6284,GALAXY,1.0,3,0.315173,1.6284,--| Based on Lyman alpha| Model fits sky resid...,3,disagreement in all. too noisy. take RR answer...
404,35185790279487153,-1,0.33.0.dev3890,0,GALAXY,1.6284,TWL,1.0,--,--,...,4.175711,1.6284,GALAXY,1.0,3,0.315173,1.6284,--| Based on Lyman alpha| Model fits sky resid...,3,disagreement in all. too noisy. take RR answer...


conflict ID =  18


In [57]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
308,35185790279488094,GALAXY,1.6137,SAE,0.0,R,--,--,strong unidentified emissions,1.6137,GALAXY,2.666667,4,1.6137,0.0,strong unidentified emissions|--|--,3,none
358,35185790279488094,GALAXY,1.6137,ACE,4.0,--,--,--,--,1.6137,GALAXY,2.666667,4,1.6137,0.0,strong unidentified emissions|--|--,3,none
408,35185790279488094,GALAXY,1.6137,TWL,4.0,--,--,--,--,1.6137,GALAXY,2.666667,4,1.6137,0.0,strong unidentified emissions|--|--,3,none


In [58]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII in skylines, but secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
308,35185790279488094,-1,0.33.0.dev3890,0,GALAXY,1.6137,SAE,3.0,R,--,...,44.628992,1.6137,GALAXY,2.666667,4,0.0,1.6137,strong unidentified emissions|--|--,3,"disagreement in flag. OII in skylines, but sec..."
358,35185790279488094,-1,0.33.0.dev3890,0,GALAXY,1.6137,ACE,3.0,--,--,...,44.628992,1.6137,GALAXY,2.666667,4,0.0,1.6137,strong unidentified emissions|--|--,3,"disagreement in flag. OII in skylines, but sec..."
408,35185790279488094,-1,0.33.0.dev3890,0,GALAXY,1.6137,TWL,3.0,--,--,...,44.628992,1.6137,GALAXY,2.666667,4,0.0,1.6137,strong unidentified emissions|--|--,3,"disagreement in flag. OII in skylines, but sec..."


conflict ID =  19


In [59]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185790279488374,GALAXY,0.9599,SAE,0.0,--,--,--,--,0.9599,GALAXY,1.0,3,0.9599,0.469301,--|--| No obvious features,3,none
361,35185790279488374,GALAXY,0.9599,ACE,3.0,R,0.3339,GALAXY,--,0.3339,GALAXY,1.0,3,0.9599,0.469301,--|--| No obvious features,3,none
411,35185790279488374,GALAXY,0.9599,TWL,0.0,--,--,--,No obvious features,0.9599,GALAXY,1.0,3,0.9599,0.469301,--|--| No obvious features,3,none


In [60]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.3339
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. VI z is not very convincing, but better than RR. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
311,35185790279488374,-1,0.33.0.dev3890,0,GALAXY,0.9599,SAE,2.0,--,--,...,10.097209,0.3339,GALAXY,1.0,3,0.469301,0.9599,--|--| No obvious features,3,disagreement in z. VI z is not very convincing...
361,35185790279488374,-1,0.33.0.dev3890,0,GALAXY,0.9599,ACE,2.0,R,0.3339,...,10.097209,0.3339,GALAXY,1.0,3,0.469301,0.9599,--|--| No obvious features,3,disagreement in z. VI z is not very convincing...
411,35185790279488374,-1,0.33.0.dev3890,0,GALAXY,0.9599,TWL,2.0,--,--,...,10.097209,0.3339,GALAXY,1.0,3,0.469301,0.9599,--|--| No obvious features,3,disagreement in z. VI z is not very convincing...


conflict ID =  20


In [61]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
312,35185790283677815,GALAXY,1.127,SAE,0.0,--,--,--,--,1.127,GALAXY,2.333333,4,1.127,0.0,--|--| [OII] on sky lines,3,none
362,35185790283677815,GALAXY,1.127,ACE,4.0,--,--,--,--,1.127,GALAXY,2.333333,4,1.127,0.0,--|--| [OII] on sky lines,3,none
412,35185790283677815,GALAXY,1.127,TWL,3.0,--,--,--,[OII] on sky lines,1.127,GALAXY,2.333333,4,1.127,0.0,--|--| [OII] on sky lines,3,none


In [62]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII on noise spikes, but secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
312,35185790283677815,-1,0.33.0.dev3890,0,GALAXY,1.127,SAE,3.0,--,--,...,51.367897,1.127,GALAXY,2.333333,4,0.0,1.127,--|--| [OII] on sky lines,3,"disagreement in flag. OII on noise spikes, but..."
362,35185790283677815,-1,0.33.0.dev3890,0,GALAXY,1.127,ACE,3.0,--,--,...,51.367897,1.127,GALAXY,2.333333,4,0.0,1.127,--|--| [OII] on sky lines,3,"disagreement in flag. OII on noise spikes, but..."
412,35185790283677815,-1,0.33.0.dev3890,0,GALAXY,1.127,TWL,3.0,--,--,...,51.367897,1.127,GALAXY,2.333333,4,0.0,1.127,--|--| [OII] on sky lines,3,"disagreement in flag. OII on noise spikes, but..."


conflict ID =  21


In [63]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
334,35185790283677860,GALAXY,1.394,SAE,0.0,--,--,--,--,1.394,GALAXY,2.0,3,1.394,0.0,--|--| [OII] next to sky lines,3,none
384,35185790283677860,GALAXY,1.394,ACE,3.0,--,--,--,--,1.394,GALAXY,2.0,3,1.394,0.0,--|--| [OII] next to sky lines,3,none
434,35185790283677860,GALAXY,1.394,TWL,3.0,--,--,--,[OII] next to sky lines,1.394,GALAXY,2.0,3,1.394,0.0,--|--| [OII] next to sky lines,3,none


In [64]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'Disagreement in flag. OII on sky line, but continuum consistent with OII. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
334,35185790283677860,-1,0.33.0.dev3890,0,GALAXY,1.394,SAE,3.0,--,--,...,103.217894,1.394,GALAXY,2.0,3,0.0,1.394,--|--| [OII] next to sky lines,3,"Disagreement in flag. OII on sky line, but con..."
384,35185790283677860,-1,0.33.0.dev3890,0,GALAXY,1.394,ACE,3.0,--,--,...,103.217894,1.394,GALAXY,2.0,3,0.0,1.394,--|--| [OII] next to sky lines,3,"Disagreement in flag. OII on sky line, but con..."
434,35185790283677860,-1,0.33.0.dev3890,0,GALAXY,1.394,TWL,3.0,--,--,...,103.217894,1.394,GALAXY,2.0,3,0.0,1.394,--|--| [OII] next to sky lines,3,"Disagreement in flag. OII on sky line, but con..."


conflict ID =  22


In [65]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
314,35185790283678333,GALAXY,1.6051,SAE,0.0,--,--,--,--,1.6051,GALAXY,2.0,3,1.6051,0.0,--|--| [OII] next to sky lines,3,none
364,35185790283678333,GALAXY,1.6051,ACE,3.0,--,--,--,--,1.6051,GALAXY,2.0,3,1.6051,0.0,--|--| [OII] next to sky lines,3,none
414,35185790283678333,GALAXY,1.6051,TWL,3.0,--,--,--,[OII] next to sky lines,1.6051,GALAXY,2.0,3,1.6051,0.0,--|--| [OII] next to sky lines,3,none


In [66]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. clear line next to skylines, but cannot be sure which.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
314,35185790283678333,-1,0.33.0.dev3890,0,GALAXY,1.6051,SAE,2.0,--,--,...,54.177735,1.6051,GALAXY,2.0,3,0.0,1.6051,--|--| [OII] next to sky lines,3,"flag disagreement, take mean. clear line next ..."
364,35185790283678333,-1,0.33.0.dev3890,0,GALAXY,1.6051,ACE,2.0,--,--,...,54.177735,1.6051,GALAXY,2.0,3,0.0,1.6051,--|--| [OII] next to sky lines,3,"flag disagreement, take mean. clear line next ..."
414,35185790283678333,-1,0.33.0.dev3890,0,GALAXY,1.6051,TWL,2.0,--,--,...,54.177735,1.6051,GALAXY,2.0,3,0.0,1.6051,--|--| [OII] next to sky lines,3,"flag disagreement, take mean. clear line next ..."


conflict ID =  23


In [67]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
336,35185790283678769,GALAXY,0.0004,SAE,4.0,C,0.0004,STAR,--,0.0004,STAR,2.333333,5,0.0004,0.0,--|White dwarf|It is a star,3,none
386,35185790283678769,GALAXY,0.0004,ACE,4.0,C,0.0004,STAR,White dwarf,0.0004,STAR,2.333333,5,0.0004,0.0,--|White dwarf|It is a star,3,none
436,35185790283678769,GALAXY,0.0004,TWL,-1.0,C,--,STAR,It is a star,0.0004,STAR,2.333333,5,0.0004,0.0,--|White dwarf|It is a star,3,none


In [68]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0004
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set a flag. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
336,35185790283678769,-1,0.33.0.dev3890,0,GALAXY,0.0004,SAE,4.0,C,0.0004,...,383.425707,0.0004,STAR,2.333333,5,0.0,0.0004,--|White dwarf|It is a star,3,one inspector forgot to set a flag. z is secure.
386,35185790283678769,-1,0.33.0.dev3890,0,GALAXY,0.0004,ACE,4.0,C,0.0004,...,383.425707,0.0004,STAR,2.333333,5,0.0,0.0004,--|White dwarf|It is a star,3,one inspector forgot to set a flag. z is secure.
436,35185790283678769,-1,0.33.0.dev3890,0,GALAXY,0.0004,TWL,4.0,C,--,...,383.425707,0.0004,STAR,2.333333,5,0.0,0.0004,--|White dwarf|It is a star,3,one inspector forgot to set a flag. z is secure.


conflict ID =  24


In [69]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
337,35185790283679657,GALAXY,0.8312,SAE,3.0,--,0.8112,GALAXY,--,0.8112,GALAXY,2.0,3,0.8312,0.011433,--|--| No obvious features,3,none
387,35185790283679657,GALAXY,0.8312,ACE,3.0,R,0.8105,GALAXY,--,0.8105,GALAXY,2.0,3,0.8312,0.011433,--|--| No obvious features,3,none
437,35185790283679657,GALAXY,0.8312,TWL,0.0,--,--,--,No obvious features,0.8312,GALAXY,2.0,3,0.8312,0.011433,--|--| No obvious features,3,none


In [70]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.811
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. VI z not secure. take VI z, flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
337,35185790283679657,-1,0.33.0.dev3890,0,GALAXY,0.8312,SAE,2.0,--,0.8112,...,6.196892,0.811,GALAXY,2.0,3,0.011433,0.8312,--|--| No obvious features,3,disagreement in flag and z. VI z not secure. t...
387,35185790283679657,-1,0.33.0.dev3890,0,GALAXY,0.8312,ACE,2.0,R,0.8105,...,6.196892,0.811,GALAXY,2.0,3,0.011433,0.8312,--|--| No obvious features,3,disagreement in flag and z. VI z not secure. t...
437,35185790283679657,-1,0.33.0.dev3890,0,GALAXY,0.8312,TWL,2.0,--,--,...,6.196892,0.811,GALAXY,2.0,3,0.011433,0.8312,--|--| No obvious features,3,disagreement in flag and z. VI z not secure. t...


conflict ID =  25


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
327,35185790283681353,QSO,4.923,SAE,4.0,RCS,0.7118,GALAXY,severe calibration issue; used OII for VIz;,0.7118,GALAXY,2.666667,4,4.923,2.4601,severe calibration issue; used OII for VIz;| ...,3,none
377,35185790283681353,QSO,4.923,ACE,4.0,RCS,0.7121,GALAXY,Poor continuum in red but lines very clear.,0.7121,GALAXY,2.666667,4,4.923,2.4601,severe calibration issue; used OII for VIz;| ...,3,none
427,35185790283681353,QSO,4.923,TWL,0.0,S,--,--,Calibration problem,4.9230,QSO,2.666667,4,4.923,2.4601,severe calibration issue; used OII for VIz;| ...,3,none


In [73]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.712
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give VI z based on secure OII. Not a QSO. Calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
327,35185790283681353,-1,0.33.0.dev3890,0,QSO,4.923,SAE,3.0,RCS,0.7118,...,146.785258,0.712,GALAXY,2.666667,4,2.4601,4.923,severe calibration issue; used OII for VIz;| ...,3,2/3 inspectors give VI z based on secure OII. ...
377,35185790283681353,-1,0.33.0.dev3890,0,QSO,4.923,ACE,3.0,RCS,0.7121,...,146.785258,0.712,GALAXY,2.666667,4,2.4601,4.923,severe calibration issue; used OII for VIz;| ...,3,2/3 inspectors give VI z based on secure OII. ...
427,35185790283681353,-1,0.33.0.dev3890,0,QSO,4.923,TWL,3.0,S,--,...,146.785258,0.712,GALAXY,2.666667,4,2.4601,4.923,severe calibration issue; used OII for VIz;| ...,3,2/3 inspectors give VI z based on secure OII. ...


conflict ID =  26


In [75]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
332,35185790283682341,GALAXY,1.5273,SAE,0.0,--,--,--,--,1.5273,GALAXY,2.0,3,1.527333,0.00004,--|--| [OII] next to sky lines,3,none
382,35185790283682341,GALAXY,1.5273,ACE,3.0,--,1.5274,GALAXY,--,1.5274,GALAXY,2.0,3,1.527333,0.00004,--|--| [OII] next to sky lines,3,none
432,35185790283682341,GALAXY,1.5273,TWL,3.0,--,--,--,[OII] next to sky lines,1.5273,GALAXY,2.0,3,1.527333,0.00004,--|--| [OII] next to sky lines,3,none


In [76]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII in skylines, but secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
332,35185790283682341,-1,0.33.0.dev3890,0,GALAXY,1.5273,SAE,3.0,--,--,...,249.646202,1.5273,GALAXY,2.0,3,0.00004,1.527333,--|--| [OII] next to sky lines,3,"disagreement in flag. OII in skylines, but sec..."
382,35185790283682341,-1,0.33.0.dev3890,0,GALAXY,1.5273,ACE,3.0,--,1.5274,...,249.646202,1.5274,GALAXY,2.0,3,0.00004,1.527333,--|--| [OII] next to sky lines,3,"disagreement in flag. OII in skylines, but sec..."
432,35185790283682341,-1,0.33.0.dev3890,0,GALAXY,1.5273,TWL,3.0,--,--,...,249.646202,1.5273,GALAXY,2.0,3,0.00004,1.527333,--|--| [OII] next to sky lines,3,"disagreement in flag. OII in skylines, but sec..."


conflict ID =  27


In [78]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
441,35185790283683134,GALAXY,0.9747,ACE,1.0,--,--,--,No clear match to weak features,0.9747,GALAXY,1.333333,3,0.9747,0.064758,No clear match to weak features|--| No obviou...,3,none
491,35185790283683134,GALAXY,0.9747,SAE,3.0,--,0.8546,GALAXY,--,0.8546,GALAXY,1.333333,3,0.9747,0.064758,No clear match to weak features|--| No obviou...,3,none
541,35185790283683134,GALAXY,0.9747,TWL,0.0,--,--,--,No obvious features,0.9747,GALAXY,1.333333,3,0.9747,0.064758,No clear match to weak features|--| No obviou...,3,none


In [79]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no clear features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
441,35185790283683134,-1,0.33.0.dev3890,0,GALAXY,0.9747,ACE,1.0,--,--,...,22.21913,0.9747,GALAXY,1.333333,3,0.064758,0.9747,No clear match to weak features|--| No obviou...,3,disagreement in flag and z. no clear features....
491,35185790283683134,-1,0.33.0.dev3890,0,GALAXY,0.9747,SAE,1.0,--,0.8546,...,22.21913,0.8546,GALAXY,1.333333,3,0.064758,0.9747,No clear match to weak features|--| No obviou...,3,disagreement in flag and z. no clear features....
541,35185790283683134,-1,0.33.0.dev3890,0,GALAXY,0.9747,TWL,1.0,--,--,...,22.21913,0.9747,GALAXY,1.333333,3,0.064758,0.9747,No clear match to weak features|--| No obviou...,3,disagreement in flag and z. no clear features....


conflict ID =  28


In [81]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
442,35185790287872074,GALAXY,0.5009,ACE,3.0,--,0.2800,GALAXY,Based on Halpha,0.2800,GALAXY,2.0,2,0.5009,0.499297,Based on Halpha|--| Possible [OII] at z~0.9184...,3,none
492,35185790287872074,GALAXY,0.5009,SAE,1.0,--,0.9191,GALAXY,--,0.9191,GALAXY,2.0,2,0.5009,0.499297,Based on Halpha|--| Possible [OII] at z~0.9184...,3,none
542,35185790287872074,GALAXY,0.5009,TWL,2.0,RS,0.9184,--,Possible [OII] at z~0.9184; Calibration featu...,0.9184,GALAXY,2.0,2,0.5009,0.499297,Based on Halpha|--| Possible [OII] at z~0.9184...,3,none


In [82]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9188
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, two VI z offered. One is more convincing but not secure, flag 2. Calibration issues. Poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
442,35185790287872074,-1,0.33.0.dev3890,0,GALAXY,0.5009,ACE,2.0,--,0.2800,...,1.153136,0.9188,GALAXY,2.0,2,0.499297,0.5009,Based on Halpha|--| Possible [OII] at z~0.9184...,3,"disagreement in z, two VI z offered. One is mo..."
492,35185790287872074,-1,0.33.0.dev3890,0,GALAXY,0.5009,SAE,2.0,--,0.9191,...,1.153136,0.9188,GALAXY,2.0,2,0.499297,0.5009,Based on Halpha|--| Possible [OII] at z~0.9184...,3,"disagreement in z, two VI z offered. One is mo..."
542,35185790287872074,-1,0.33.0.dev3890,0,GALAXY,0.5009,TWL,2.0,RS,0.9184,...,1.153136,0.9188,GALAXY,2.0,2,0.499297,0.5009,Based on Halpha|--| Possible [OII] at z~0.9184...,3,"disagreement in z, two VI z offered. One is mo..."


conflict ID =  29


In [85]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
445,35185790287872604,GALAXY,0.1982,ACE,4.0,--,--,--,--,0.1982,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,none
495,35185790287872604,GALAXY,0.1982,SAE,1.0,R,1.1094,GALAXY,--,1.1094,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,none
545,35185790287872604,GALAXY,0.1982,TWL,4.0,--,--,--,--,0.1982,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,none


In [87]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag and z disagreement, but RR z is secure. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
445,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,ACE,4.0,--,--,...,184.267594,0.1982,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."
495,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,SAE,4.0,R,1.1094,...,184.267594,1.1094,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."
545,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,TWL,4.0,--,--,...,184.267594,0.1982,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."


conflict ID =  30


In [89]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
472,35185790292066750,GALAXY,1.4102,ACE,3.0,--,--,--,--,1.4102,GALAXY,2.0,2,1.4102,0.0,--|--| Weak [OII] on sky lines,3,none
522,35185790292066750,GALAXY,1.4102,SAE,1.0,--,--,--,--,1.4102,GALAXY,2.0,2,1.4102,0.0,--|--| Weak [OII] on sky lines,3,none
572,35185790292066750,GALAXY,1.4102,TWL,2.0,--,--,--,Weak [OII] on sky lines,1.4102,GALAXY,2.0,2,1.4102,0.0,--|--| Weak [OII] on sky lines,3,none


In [90]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
472,35185790292066750,-1,0.33.0.dev3890,0,GALAXY,1.4102,ACE,2.0,--,--,...,6.99939,1.4102,GALAXY,2.0,2,0.0,1.4102,--|--| Weak [OII] on sky lines,3,"flag disagreement, take mean."
522,35185790292066750,-1,0.33.0.dev3890,0,GALAXY,1.4102,SAE,2.0,--,--,...,6.99939,1.4102,GALAXY,2.0,2,0.0,1.4102,--|--| Weak [OII] on sky lines,3,"flag disagreement, take mean."
572,35185790292066750,-1,0.33.0.dev3890,0,GALAXY,1.4102,TWL,2.0,--,--,...,6.99939,1.4102,GALAXY,2.0,2,0.0,1.4102,--|--| Weak [OII] on sky lines,3,"flag disagreement, take mean."


conflict ID =  31


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
599,35185790292071269,GALAXY,1.5901,SAE,0.0,--,--,--,--,1.5901,GALAXY,2.666667,4,1.5907,0.000386,--| CIV in emission?|--,3,none
646,35185790292071269,GALAXY,1.5901,ACE,4.0,--,1.5909,GALAXY,CIV in emission?,1.5909,GALAXY,2.666667,4,1.5907,0.000386,--| CIV in emission?|--,3,none
696,35185790292071269,GALAXY,1.5901,TWL,4.0,--,1.5911,--,--,1.5911,GALAXY,2.666667,4,1.5907,0.000386,--| CIV in emission?|--,3,none


In [93]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.591
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but z is secure. small z adjustement as template profile is not right.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
599,35185790292071269,-1,0.33.0.dev3890,0,GALAXY,1.5901,SAE,3.0,--,--,...,27.689388,1.591,GALAXY,2.666667,4,0.000386,1.5907,--| CIV in emission?|--,3,flag disagreement but z is secure. small z adj...
646,35185790292071269,-1,0.33.0.dev3890,0,GALAXY,1.5901,ACE,3.0,--,1.5909,...,27.689388,1.591,GALAXY,2.666667,4,0.000386,1.5907,--| CIV in emission?|--,3,flag disagreement but z is secure. small z adj...
696,35185790292071269,-1,0.33.0.dev3890,0,GALAXY,1.5901,TWL,3.0,--,1.5911,...,27.689388,1.591,GALAXY,2.666667,4,0.000386,1.5907,--| CIV in emission?|--,3,flag disagreement but z is secure. small z adj...


conflict ID =  32


In [95]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
619,35185790296260657,QSO,1.038,SAE,3.0,RS,0.6932,QSO,--,0.6932,QSO,2.333333,4,1.038,0.203638,--|Problem with subtraction in infrared band. ...,3,none
667,35185790296260657,QSO,1.038,ACE,4.0,RS,0.6932,GALAXY,Problem with subtraction in infrared band. Cle...,0.6932,GALAXY,2.333333,4,1.038,0.203638,--|Problem with subtraction in infrared band. ...,3,none
717,35185790296260657,QSO,1.038,TWL,0.0,CS,--,--,Bad calibration features,1.0380,QSO,2.333333,4,1.038,0.203638,--|Problem with subtraction in infrared band. ...,3,none


In [96]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6932
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'RR is incorrect, not a QSO. Calibration issues. 2/3 inspectors give VI z based on clear broadened line, but not clear that is OII. Flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
619,35185790296260657,-1,0.33.0.dev3890,0,QSO,1.038,SAE,2.0,RS,0.6932,...,79.514057,0.6932,GALAXY,2.333333,4,0.203638,1.038,--|Problem with subtraction in infrared band. ...,3,"RR is incorrect, not a QSO. Calibration issues..."
667,35185790296260657,-1,0.33.0.dev3890,0,QSO,1.038,ACE,2.0,RS,0.6932,...,79.514057,0.6932,GALAXY,2.333333,4,0.203638,1.038,--|Problem with subtraction in infrared band. ...,3,"RR is incorrect, not a QSO. Calibration issues..."
717,35185790296260657,-1,0.33.0.dev3890,0,QSO,1.038,TWL,2.0,CS,--,...,79.514057,0.6932,GALAXY,2.333333,4,0.203638,1.038,--|Problem with subtraction in infrared band. ...,3,"RR is incorrect, not a QSO. Calibration issues..."


conflict ID =  33


In [98]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
673,35185790296262065,GALAXY,1.5677,ACE,4.0,--,1.5677,QSO,--,1.5677,QSO,3.5,1,1.5677,0.0,--| Triple peak [OII] next to sky lines,2,none
723,35185790296262065,GALAXY,1.5677,TWL,3.0,--,--,--,Triple peak [OII] next to sky lines,1.5677,GALAXY,3.5,1,1.5677,0.0,--| Triple peak [OII] next to sky lines,2,none


In [99]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype, not a QSO. Clear broadened line but not clear what it is. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
673,35185790296262065,-1,0.33.0.dev3890,0,GALAXY,1.5677,ACE,2.0,--,1.5677,...,18.437941,1.5677,GALAXY,3.5,1,0.0,1.5677,--| Triple peak [OII] next to sky lines,2,"disagreement in spectype, not a QSO. Clear bro..."
723,35185790296262065,-1,0.33.0.dev3890,0,GALAXY,1.5677,TWL,2.0,--,--,...,18.437941,1.5677,GALAXY,3.5,1,0.0,1.5677,--| Triple peak [OII] next to sky lines,2,"disagreement in spectype, not a QSO. Clear bro..."


conflict ID =  34


In [101]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
607,35185790296263776,GALAXY,1.4978,SAE,1.0,R,--,--,--,1.4978,GALAXY,1.333333,3,1.4981,0.00036,--| MgII in absorption but OII on bright sky l...,3,none
655,35185790296263776,GALAXY,1.4978,ACE,3.0,--,1.4987,GALAXY,MgII in absorption but OII on bright sky lime/,1.4987,GALAXY,1.333333,3,1.4981,0.00036,--| MgII in absorption but OII on bright sky l...,3,none
705,35185790296263776,GALAXY,1.4978,TWL,0.0,RS,--,--,Model fits sky residual; calibration problem,1.4978,GALAXY,1.333333,3,1.4981,0.00036,--| MgII in absorption but OII on bright sky l...,3,none


In [102]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
607,35185790296263776,-1,0.33.0.dev3890,0,GALAXY,1.4978,SAE,1.333333,R,--,...,80.650014,1.4978,GALAXY,1.333333,3,0.00036,1.4981,--| MgII in absorption but OII on bright sky l...,3,"flag disagreement, take mean. z not secure."
655,35185790296263776,-1,0.33.0.dev3890,0,GALAXY,1.4978,ACE,1.333333,--,1.4987,...,80.650014,1.4987,GALAXY,1.333333,3,0.00036,1.4981,--| MgII in absorption but OII on bright sky l...,3,"flag disagreement, take mean. z not secure."
705,35185790296263776,-1,0.33.0.dev3890,0,GALAXY,1.4978,TWL,1.333333,RS,--,...,80.650014,1.4978,GALAXY,1.333333,3,0.00036,1.4981,--| MgII in absorption but OII on bright sky l...,3,"flag disagreement, take mean. z not secure."


conflict ID =  35


In [104]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
608,35185790296263840,GALAXY,0.4955,SAE,0.0,R,--,--,--,0.4955,GALAXY,1.0,3,0.4955,0.220662,--| Very weak OII but MgII in emission and Hde...,3,none
656,35185790296263840,GALAXY,0.4955,ACE,3.0,R,0.8255,GALAXY,Very weak OII but MgII in emission and Hdelta...,0.8255,GALAXY,1.0,3,0.4955,0.220662,--| Very weak OII but MgII in emission and Hde...,3,none
706,35185790296263840,GALAXY,0.4955,TWL,0.0,S,--,--,Calibration problem; no obvious features,0.4955,GALAXY,1.0,3,0.4955,0.220662,--| Very weak OII but MgII in emission and Hde...,3,none


In [105]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4955
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. RR answerr, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
608,35185790296263840,-1,0.33.0.dev3890,0,GALAXY,0.4955,SAE,1.0,R,--,...,33.764645,0.4955,GALAXY,1.0,3,0.220662,0.4955,--| Very weak OII but MgII in emission and Hde...,3,disagreement in flag and z. no secure features...
656,35185790296263840,-1,0.33.0.dev3890,0,GALAXY,0.4955,ACE,1.0,R,0.8255,...,33.764645,0.4955,GALAXY,1.0,3,0.220662,0.4955,--| Very weak OII but MgII in emission and Hde...,3,disagreement in flag and z. no secure features...
706,35185790296263840,-1,0.33.0.dev3890,0,GALAXY,0.4955,TWL,1.0,S,--,...,33.764645,0.4955,GALAXY,1.0,3,0.220662,0.4955,--| Very weak OII but MgII in emission and Hde...,3,disagreement in flag and z. no secure features...


conflict ID =  36


In [107]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
610,35185790296264034,GALAXY,0.5026,SAE,0.0,--,--,--,--,0.5026,GALAXY,2.666667,4,0.5026,0.343338,--|Redrock confuses OII with OIII. MgII in abs...,3,none
658,35185790296264034,GALAXY,0.5026,ACE,4.0,R,1.0185,GALAXY,Redrock confuses OII with OIII. MgII in absorp...,1.0185,GALAXY,2.666667,4,0.5026,0.343338,--|Redrock confuses OII with OIII. MgII in abs...,3,none
708,35185790296264034,GALAXY,0.5026,TWL,4.0,R,1.0183,--,Calibration features drive the best-fit,1.0183,GALAXY,2.666667,4,0.5026,0.343338,--|Redrock confuses OII with OIII. MgII in abs...,3,none


In [108]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0183
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 2/3 inspectors give same VI z based on weak but secure OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
610,35185790296264034,-1,0.33.0.dev3890,0,GALAXY,0.5026,SAE,3.0,--,--,...,16.470338,1.0183,GALAXY,2.666667,4,0.343338,0.5026,--|Redrock confuses OII with OIII. MgII in abs...,3,disagreement in z. 2/3 inspectors give same VI...
658,35185790296264034,-1,0.33.0.dev3890,0,GALAXY,0.5026,ACE,3.0,R,1.0185,...,16.470338,1.0183,GALAXY,2.666667,4,0.343338,0.5026,--|Redrock confuses OII with OIII. MgII in abs...,3,disagreement in z. 2/3 inspectors give same VI...
708,35185790296264034,-1,0.33.0.dev3890,0,GALAXY,0.5026,TWL,3.0,R,1.0183,...,16.470338,1.0183,GALAXY,2.666667,4,0.343338,0.5026,--|Redrock confuses OII with OIII. MgII in abs...,3,disagreement in z. 2/3 inspectors give same VI...


conflict ID =  37


### and so on...

We should now recompute the conflicts, and not find any.

In [109]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [110]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185790287872604]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [111]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
445,35185790287872604,GALAXY,0.1982,ACE,4.0,--,--,--,--,0.1982,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,"flag and z disagreement, but RR z is secure."
495,35185790287872604,GALAXY,0.1982,SAE,4.0,R,1.1094,GALAXY,--,1.1094,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,"flag and z disagreement, but RR z is secure."
545,35185790287872604,GALAXY,0.1982,TWL,4.0,--,--,--,--,0.1982,GALAXY,3.0,3,0.1982,0.760474,--|--|--,3,"flag and z disagreement, but RR z is secure."


In [112]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.1982
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
445,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,ACE,4.0,--,--,...,184.267594,0.1982,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."
495,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,SAE,4.0,R,1.1094,...,184.267594,0.1982,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."
545,35185790287872604,-1,0.33.0.dev3890,0,GALAXY,0.1982,TWL,4.0,--,--,...,184.267594,0.1982,GALAXY,3.0,3,0.760474,0.1982,--|--|--,3,"flag and z disagreement, but RR z is secure."


conflict ID =  0


Let's check the number of conflicts again.

In [113]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [114]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [115]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [116]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [117]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets41_45.txt


In [118]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [119]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [120]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [122]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185784260661713,1.1575,GALAXY,1.1575,GALAXY,4.000000,3,Triple peak [OII]|--|--,One inspector forgot to set the flag. z is sec...,634.645283,3807,0.829419,1.421824,5.120096,0.386846,0.663147,2.388042
1,35185784260661795,1.1640,GALAXY,1.1640,GALAXY,3.666667,3,--|--|--,none,133.904435,3846,0.230305,0.260803,0.494600,0.179159,0.202884,0.384760
2,35185784260662258,0.8274,GALAXY,0.8274,GALAXY,3.666667,3,--|--|--,none,204.922508,3944,0.377497,0.432355,0.532461,0.115302,0.132057,0.162634
3,35185784264851762,1.0654,GALAXY,1.0654,GALAXY,4.000000,3,--|--|--,none,2383.240028,3835,1.264859,1.683720,3.402119,0.848855,1.129956,2.283185
4,35185784264852063,0.8567,GALAXY,0.8567,GALAXY,4.000000,3,Triple peak [OII]|--|--,none,6237.108686,3922,1.106601,1.691944,3.785463,0.645052,0.986257,2.206596
5,35185784264852150,0.8273,GALAXY,0.8273,GALAXY,4.000000,3,--|--|--,none,1411.748749,3916,0.373507,0.646221,1.533835,0.186597,0.322841,0.766277
6,35185784264852269,0.9185,GALAXY,0.9185,GALAXY,4.000000,3,--|--|--,none,158.407160,3947,0.525037,0.908206,2.692697,0.173149,0.299512,0.888010
7,35185784264852301,0.9808,GALAXY,0.9808,GALAXY,4.000000,3,--|--|--,none,126.361543,3848,0.258360,0.380436,0.858155,0.201002,0.295976,0.667637
8,35185784264852520,1.0662,GALAXY,1.0662,GALAXY,4.000000,3,--|--|--,none,1567.598254,3924,1.090219,1.247429,2.095578,0.670753,0.767476,1.289297
9,35185784264852626,0.7687,GALAXY,0.7687,GALAXY,1.000000,3,Model is driven by calibration features| Weak...,disagreement in flag and z. no clear features....,0.099587,3933,0.434014,0.448127,0.492440,0.187175,0.193262,0.212373


In [123]:
merged_file.loc[9]['merger comment']

'disagreement in flag and z. no clear features. take RR answer, flag 1.'

In [124]:
merged_file.loc[9]['all VI comments']

' Model is driven by calibration features| Weak OII and Hgamma and Hbeta|--'

In [125]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'